## Imports

In [ ]:
import osmnx as ox
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, Polygon, LineString
from geopy.geocoders import Nominatim
from geopandas import GeoDataFrame
import networkx as nx
import spaghetti
import esda
import random
from scipy.spatial import distance
from itertools import combinations
import datetime
from prov import *
from prov.dot import prov_to_dot
from prov.model import ProvDocument
from IPython.display import Image
from prov.model import ProvRecord
import seaborn as sns

## Task A

In [ ]:
# Step 2: Download road network data for Bond Street area in Leeds
place_name = "Bond Street, Leeds, UK"
graph = ox.graph_from_place(place_name, network_type="drive", buffer_dist=1000)

In [ ]:
# Create a geolocator object
geolocator = Nominatim(user_agent="my_app")

# Get the location object for the Bond Street area in Leeds
location = geolocator.geocode(place_name)

latitude = location.latitude
longitude = location.longitude

# Print the latitude and longitude coordinates
print(location.latitude, location.longitude)

In [ ]:
# Filter accident data to only include accidents within Bond Street area
# Get the geographical area of bond street area from OpenStreetMap as a GeoDataFrame
bondstreet_area = ox.geocode_to_gdf(place_name, buffer_dist=1000)
# From this take the polygon that defines Bond's boundary
bond_polygon = bondstreet_area['geometry'][0]
bond_polygon

In [ ]:
# Load accident data for 2018 and 2019
ac_2018 = pd.read_csv("accidents_2018.csv")
ac_2019 = pd.read_csv("accidents_2019.csv")

In [ ]:
#Concat both pandas dataframe
accidents = pd.concat([ac_2018, ac_2019], axis=0)

# Define the projection for the British National Grid
bng = pyproj.Proj(init="epsg:27700")

# Define the projection for latitude and longitude
wgs84 = pyproj.Proj(init="epsg:4326")

#Obtain latitude and longitude of the accidents in the dataset
accidents["Longitude"], accidents["Latitude"] = pyproj.transform(bng, wgs84, accidents["Grid Ref: Easting"].values, accidents["Grid Ref: Northing"].values)

In [ ]:
# Translate to a GeoDatFrame where the geometry is given by a Point constructed from the longitude and latitude
all_accidents = GeoDataFrame(geometry=[Point(xy) for xy in zip(
    accidents.Longitude, accidents.Latitude)])

In [ ]:
all_accidents

In [ ]:
# Filter the crime points to just those within Bond Street, as defined by the polygon found earlier
accidents_within_area = all_accidents[all_accidents.geometry.within(bond_polygon)]
accidents_within_area

In [ ]:
leeds_network = ox.graph_from_polygon(bond_polygon, network_type='drive')
ox.plot_graph(leeds_network)

In [ ]:
ox.plot_graph_folium(leeds_network)

In [ ]:
# This line of code calculates the area of the Bond Street region in Leeds in square meters, which can be used to compute various
# statistics related to the road network and accident data in the area.
bondstreet_area = ox.project_gdf(bondstreet_area).unary_union.area
stats = ox.basic_stats(leeds_network, area=bondstreet_area)

In [ ]:
ox.extended_states(leeds_network, ecc=True)['diameter']

In [ ]:
stats

In [ ]:
is_planar, kuratowski_subgraphs = nx.check_planarity(leeds_network, counterexample=True)
is_planar

In [ ]:
plt.figure(figsize=(18, 8))
nx.draw(kuratowski_subgraphs, pos=nx.kamada_kawai_layout(kuratowski_subgraphs), arrows=True, arrowstyle='-|>', with_labels=True)
plt.show()

## Task B

In [ ]:
# Some edges (roads in Bond street) are missing a geometry, so we will create one
# assuming that the road is a direct line from start point to end point.
# First, we need to get the locations of the points
x_values = nx.get_node_attributes(leeds_network, 'x')
y_values = nx.get_node_attributes(leeds_network, 'y')

# We will work with the edges and add the missing geometries (lines denoting the roads between points)
graph_with_geometries = list(leeds_network.edges(data=True))

# Iterate through the edges and, where missing, add a geometry attribute with the line between start and end nodes
for e in graph_with_geometries:
    if not 'geometry' in e[2]:
        e[2]['geometry'] = LineString([
            Point(x_values[e[0]], y_values[e[0]]),
            Point(x_values[e[1]], y_values[e[1]])])

In [ ]:
# We will now drop the start and end nodes, as we will construct a new Spaghetti network based on the geometries of the roads
road_lines = [x[2] for x in graph_with_geometries]
# From this, we can construct a GeoDataFrame
roads_geodataframe = GeoDataFrame(pd.DataFrame(road_lines))
roads_geodataframe

In [ ]:
# From the GeoDataFrame, we can construct a network in Spaghetti from which to do point analysis
bond_points_graph = spaghetti.Network(in_data=roads_geodataframe)

In [ ]:
# Check what this new network looks like by getting DataFrames for the edges and plotting them
nodes_df, edges_df = spaghetti.element_as_gdf(
    bond_points_graph, vertices=True, arcs=True)

base_network = edges_df.plot(color = "k", zorder=0, figsize=(15, 15))
nodes_df.plot(ax=base_network, color="r", zorder=2)

In [ ]:
# We will now snap the bond street located accidents we extracted earlier, 
#i.e. position them at the closest point on the closest road
bond_points_graph.snapobservations(accidents_within_area, 'accidents')

In [ ]:
# Show the network
base_network = edges_df.plot(color="k", zorder=0, figsize =(12, 12))
# Get a GeoDataFrame of the snapped accident locations to plot on the network image
snapped_accidents=spaghetti.element_as_gdf(
    bond_points_graph, pp_name='accidents', snapped=True)

# Plot these on the road network
snapped_accidents.plot(
    color="r", marker="x",
    markersize=50, zorder=1, ax=base_network)

plt.show()

In [ ]:
# Show the network
base_network = edges_df.plot(color="k", zorder=0, figsize =(12, 12))
# Get a GeoDataFrame of the accident locations to plot on the network image
snapped_accidents=spaghetti.element_as_gdf(
    bond_points_graph, pp_name='accidents', snapped=False)

# Plot these on the road network
snapped_accidents.plot(
    color="r", marker="x",
    markersize=50, zorder=1, ax=base_network)

plt.show()

In [ ]:
kres = bond_points_graph.GlobalAutoK(
    bond_points_graph.pointpatterns["accidents"],
    nsteps=50, permutations=100
)

In [ ]:
kres.lam
kres.xaxis
kres.observed
kres.upperenvelope
kres.lowerenvelope
kres.sim

print(f"Density of points in the network (lambda): {kres.lam}")

In [ ]:
print(f"Distances at which density is measured:\n{kres.xaxis}")

In [ ]:
fig, ax = plt.subplots()

ax.plot(kres.xaxis, kres.observed, "b-", label="Observed")
ax.plot(kres.xaxis, kres.upperenvelope, "r--", label="Upper")
ax.plot(kres.xaxis, kres.lowerenvelope, "k--", label="Lower")

ax.legend(loc="best", fontsize="x-large")
ax.set_xlabel("Distance $(r)$")
ax.set_ylabel("$K(r)$")

fig.tight_layout()

In [ ]:
# Get snapped point pattern 
pointpat = bond_points_graph.pointpatterns['accidents']
# Get count of points per network edge: a dictionary from each edge to the crime count on that edge
counts = bond_points_graph.count_per_link(pointpat.obs_to_arc, graph=False)

In [ ]:
# Get the weights matrix for edges in the graph (just the adjacency matrix with 1 where edges connect at a node, 0 otherwise)
weights = bond_points_graph.w_network

In [ ]:
# Get the edges included in the weights matrix: an enumerator for a list of edges
edges = weights.neighbors.keys()
# Construct an array of the counts values per edge in the same order as
# the weights matrix, with 0.0 where no counts recorded
values = [counts[edge] if edge in counts.keys () else 0. \
    for index, edge in enumerate(edges)]

In [ ]:
moran = esda.moran.Moran(values, weights)
moran.I

In [ ]:
moran.p_sim

In [ ]:
moran.EI
print(moran.EI_sim)

print(moran.EI)

In [ ]:
moran.sim

In [ ]:
sns.kdeplot(moran.sim, shade=True)
plt.vlines(moran.I, 0, 1, color='r')
plt.vlines(moran.EI, 0,1)
plt.xlabel("Moran's I")

## Task C

In [ ]:
# Define the query place and get the graph
query_place = "Leeds, UK"
G = ox.graph_from_place(query_place, network_type="drive")

# Get the node coordinates as a numpy array
coords = np.array([[data["x"], data["y"]] for node, data in G.nodes(data=True)])

# Choose the number of clusters
k = 10

# Run the k-means clustering algorithm
kmeans = KMeans(n_clusters=k, random_state=0).fit(coords)

# Get the centroids of the clusters as the seed points
seeds = kmeans.cluster_centers_

# Find the nearest node to each centroid and  it to the seed list
closest_nodes = [ox.distance.nearest_nodes(G, seed[0], seed[1]) for seed in seeds]
seeds = [min(closest_nodes, key=lambda node: np.linalg.norm(np.array([G.nodes[node]['x'], G.nodes[node]['y']]) - seed)) for seed in seeds]

In [ ]:
# Black color representation
black_color = (0.0, 0.0, 0.0, 1.0)  # change 'k' to RGBs value tuple (0.0, 0.0, 0.0, 1.0) if ValueError
all_nodes = list(G.nodes)

In [ ]:
distances = {seed: nx.single_source_dijkstra_path_length(
    G, seed, weight='length') for seed in seeds}

def nearest_from_list(node_distances):
    return sorted(node_distances, key=lambda node_length: node_length[1])[0] \
        if len(node_distances) > 0 else None

def nearest_seed(node):
    seed_distances = [(seed, distances[seed][node]) \
        for seed in seeds if node in distances[seed]]
    return nearest_from_list(seed_distances)

def nearest_for_edge(edge):
    nearest_to_ends_all = [nearest_seed(edge[0]), nearest_seed(edge[1])]
    nearest_to_ends = [distance for distance in nearest_to_ends_all if distance]
    return nearest_from_list(nearest_to_ends)

colours = ox.plot.get_colors(k)

def colour_for_seed_distance(seed):
    return colours[seeds.index(seed[0])]

edge_nearest_seeds = [nearest_for_edge(edge) for edge in G.edges]
# Note that edges not connected to a seed shown in black, so invisible on black background
edge_colours = [colour_for_seed_distance(seed) if seed else black_color for seed in edge_nearest_seeds]  
# For the road network nodes, we want the seeds to be coloured red and the non-seed nodes to be coloured white.
node_colours = ['r' if node in seeds else 'w' for node in all_nodes]

ox.plot.plot_graph(G, edge_color = edge_colours, node_color = node_colours, bgcolor = 'k', save = True, filepath = 'nvd.png')

In [ ]:
def get_nodes_by_seed():
    nodes_by_seed = {seed: [] for seed in seeds}
    
    for node in all_nodes:
        nearest = nearest_seed(node)
        if nearest:
            nodes_by_seed[nearest[0]].append(node)
    
    return nodes_by_seed

# Get the nodes that belong to each cell seed point
nodes_by_seed = get_nodes_by_seed()


In [ ]:
def marathon_paths(graph, voronoi_dict, target_length=42000, tolerance=250):
    paths_42 = {}
    
    for seed, cell_nodes in voronoi_dict.items():
        subgraph = graph.subgraph(cell_nodes)
        undirected_simple_subgraph = nx.Graph(subgraph.to_undirected())
        all_cycle_seed = nx.cycle_basis(undirected_simple_subgraph)
        
        cell_cycles = []
        for cycle in all_cycle_seed:
            cycle_length = 0
            for i in range(len(cycle) - 1):
                if cycle[i+1] in subgraph[cycle[i]]:
                    cycle_length += subgraph[cycle[i]][cycle[i+1]][0]['length']
            if cycle[0] in subgraph[cycle[-1]]:
                cycle_length += subgraph[cycle[-1]][cycle[0]][0]['length']
            
            if target_length - tolerance <= cycle_length <= target_length + tolerance:
                cell_cycles.append((cycle, cycle_length))
                print("Path found of length {} meters for seed {}".format(seed, cycle_length))
        
        paths_42[seed] = cell_cycles
        print(paths_42)
    
    return paths_42

def plot_subgraph_with_path(G, subgraph_nodes, seed, path):
    subgraph = G.subgraph(subgraph_nodes).copy()
    edge_colours = [colour_for_seed_distance(nearest_for_edge(edge)) if nearest_for_edge(edge) else black_color for edge in subgraph.edges]
    node_colours = ['r' if node in seeds else 'w' for node in subgraph_nodes]

    # Customize the plot appearance
    edge_linewidths = [2 if (u, v) in path_edges or (v, u) in path_edges else 1 for u, v in subgraph.edges()]
    path_edge_colors = ['red' if (u, v) in path_edges or (v, u) in path_edges else edge_colours[i] for i, (u, v) in enumerate(subgraph.edges())]
    node_sizes = [50 if node == seed else 15 for node in subgraph_nodes]

    fig, ax = ox.plot_graph(subgraph, bgcolor='k', node_color=node_colours, node_size=node_sizes,
                             node_zorder=2, edge_color=path_edge_colors, edge_linewidth=edge_linewidths,
                             edge_alpha=1, dpi=100, close=True)
    
paths_42 = marathon_paths(G, nodes_by_seed)

for seed, paths in paths_42.items():
    if paths:  # If there's a path found for the seed point
        path_edges = [(paths[0][0][i], paths[0][0][i + 1]) for i in range(len(paths[0][0]) - 1)] + [(paths[0][0][-1], paths[0][0][0])]
        plot_subgraph_with_path(G, nodes_by_seed[seed], seed, path_edges)

# Part 5 task C

In [ ]:
# Define the query place and get the graph
query_place = "Leeds, UK"
G = ox.graph_from_place(query_place, network_type="drive")

# Get the node coordinates as a numpy array
coords = np.array([[data["x"], data["y"]] for node, data in G.nodes(data=True)])

# Choose the number of clusters
k = 7

# Run the k-means clustering algorithm
kmeans = KMeans(n_clusters=k, random_state=0).fit(coords)

# Get the centroids of the clusters as the seed points
seeds = kmeans.cluster_centers_

# Find the nearest node to each centroid and  it to the seed list
closest_nodes = [ox.distance.nearest_nodes(G, seed[0], seed[1]) for seed in seeds]
seeds = [min(closest_nodes, key=lambda node: np.linalg.norm(np.array([G.nodes[node]['x'], G.nodes[node]['y']]) - seed)) for seed in seeds]

# Black color representation
black_color = (0.0, 0.0, 0.0, 1.0)  # change 'k' to RGBs value tuple (0.0, 0.0, 0.0, 1.0) if ValueError
all_nodes = list(G.nodes)

distances = {seed: nx.single_source_dijkstra_path_length(
    G, seed, weight='length') for seed in seeds}

def nearest_from_list(node_distances):
    return sorted(node_distances, key=lambda node_length: node_length[1])[0] \
        if len(node_distances) > 0 else None

def nearest_seed(node):
    seed_distances = [(seed, distances[seed][node]) \
        for seed in seeds if node in distances[seed]]
    return nearest_from_list(seed_distances)

def nearest_for_edge(edge):
    nearest_to_ends_all = [nearest_seed(edge[0]), nearest_seed(edge[1])]
    nearest_to_ends = [distance for distance in nearest_to_ends_all if distance]
    return nearest_from_list(nearest_to_ends)

colours = ox.plot.get_colors(k)

def colour_for_seed_distance(seed):
    return colours[seeds.index(seed[0])]

edge_nearest_seeds = [nearest_for_edge(edge) for edge in G.edges]
# Note that edges not connected to a seed shown in black, so invisible on black background
edge_colours = [colour_for_seed_distance(seed) if seed else black_color for seed in edge_nearest_seeds]  
# For the road network nodes, we want the seeds to be coloured red and the non-seed nodes to be coloured white.
node_colours = ['r' if node in seeds else 'w' for node in all_nodes]

ox.plot.plot_graph(G, edge_color = edge_colours, node_color = node_colours, bgcolor = 'k', save = True, filepath = 'nvd.png')

In [ ]:
def get_nodes_by_seed():
    nodes_by_seed = {seed: [] for seed in seeds}
    
    for node in all_nodes:
        nearest = nearest_seed(node)
        if nearest:
            nodes_by_seed[nearest[0]].append(node)
    
    return nodes_by_seed

# Get the nodes that belong to each cell seed point
nodes_by_seed = get_nodes_by_seed()

def marathon_paths(graph, voronoi_dict, target_length=42000, tolerance=500):
    paths_42 = {}
    
    for seed, cell_nodes in voronoi_dict.items():
        subgraph = graph.subgraph(cell_nodes)
        undirected_simple_subgraph = nx.Graph(subgraph.to_undirected())
        all_cycle_seed = nx.cycle_basis(undirected_simple_subgraph)
        
        cell_cycles = []
        for cycle in all_cycle_seed:
            cycle_length = 0
            for i in range(len(cycle) - 1):
                if cycle[i+1] in subgraph[cycle[i]]:
                    cycle_length += subgraph[cycle[i]][cycle[i+1]][0]['length']
            if cycle[0] in subgraph[cycle[-1]]:
                cycle_length += subgraph[cycle[-1]][cycle[0]][0]['length']
            
            if target_length - tolerance <= cycle_length <= target_length + tolerance:
                cell_cycles.append((cycle, cycle_length))
                print("Path found of length {} meters for seed {}".format(seed, cycle_length))
        
        paths_42[seed] = cell_cycles
        print(paths_42)
    
    return paths_42

def plot_subgraph_with_path(G, subgraph_nodes, seed, path):
    subgraph = G.subgraph(subgraph_nodes).copy()
    edge_colours = [colour_for_seed_distance(nearest_for_edge(edge)) if nearest_for_edge(edge) else black_color for edge in subgraph.edges]
    node_colours = ['r' if node in seeds else 'w' for node in subgraph_nodes]

    # Customize the plot appearance
    edge_linewidths = [2 if (u, v) in path_edges or (v, u) in path_edges else 1 for u, v in subgraph.edges()]
    path_edge_colors = ['red' if (u, v) in path_edges or (v, u) in path_edges else edge_colours[i] for i, (u, v) in enumerate(subgraph.edges())]
    node_sizes = [50 if node == seed else 15 for node in subgraph_nodes]

    fig, ax = ox.plot_graph(subgraph, bgcolor='k', node_color=node_colours, node_size=node_sizes,
                             node_zorder=2, edge_color=path_edge_colors, edge_linewidth=edge_linewidths,
                             edge_alpha=1, dpi=100, close=True)
    
paths_42 = marathon_paths(G, nodes_by_seed)

for seed, paths in paths_42.items():
    if paths:  # If there's a path found for the seed point
        path_edges = [(paths[0][0][i], paths[0][0][i + 1]) for i in range(len(paths[0][0]) - 1)] + [(paths[0][0][-1], paths[0][0][0])]
        plot_subgraph_with_path(G, nodes_by_seed[seed], seed, path_edges)

## Task D

In [ ]:
# Create a new provenance document
doc = ProvDocument()

# Set the namespace prefixes
doc.add_namespace('dataset2018', 'https://datamillnorth.org/download/road-traffic-accidents/8c100249-09c5-4aac-91c1-9c7c3656892b/RTC%25202018_Leeds.csv')
doc.add_namespace('dataset2019', 'https://datamillnorth.org/download/road-traffic-accidents/8e6585f6-e627-4258-b16f-ca3858c0cc67/Traffic%2520accidents_2019_Leeds.csv')
doc.add_namespace('accident', 'https://www.data.gov.uk/dataset/6efe5505-941f-45bf-b576-4c1e09b579a1/road-traffic-accidents')

# Define the entities
dataset2018 = doc.entity('dataset2018:accidents', {'prov:type': 'dataset', 'prov:location': 'Leeds'})
dataset2019 = doc.entity('dataset2019:accidents', {'prov:type': 'dataset', 'prov:location': 'Leeds'})
graph = doc.entity('accident:graph', {'prov:type': 'graph','prov:location': ' Bond Street, Leeds'})

# Define the activities
gather_data_2018 = doc.activity('dataset2018:gather-data')
gather_data_2019 = doc.activity('dataset2019:gather-data')
create_graph = doc.activity('accident:create_graph-data')


# Define the agents
data_collector_2018 = doc.agent('dataset2018:data-collector', {'prov:type': 'person'})
data_collector_2019 = doc.agent('dataset2019:data-collector', {'prov:type': 'person'})
data_analyst_2018 = doc.agent('dataset2018:data-analyst', {'prov:type': 'person'})
data_analyst_2019 = doc.agent('dataset2019:data-analyst', {'prov:type': 'person'})



# Define the relationships between entities, activities, and agents
doc.wasGeneratedBy(dataset2018, gather_data_2018)
doc.wasGeneratedBy(dataset2019, gather_data_2019)
doc.used(create_graph, dataset2018)
doc.used(create_graph, dataset2019)
doc.wasGeneratedBy(graph, create_graph)
doc.wasAttributedTo(gather_data_2018, data_collector_2018)
doc.wasAttributedTo(gather_data_2019, data_collector_2019)
doc.wasAttributedTo(create_graph, data_analyst_2018)
doc.wasAttributedTo(create_graph, data_analyst_2019)

In [ ]:
# visualize the graph
dot = prov_to_dot(doc)
dot.write_png('diagram.png')

In [ ]:
Image('diagram.png')

In [ ]:
with open('prov_document.prov', 'w') as f:
    prov_serialization = doc.serialize(format='json')
    f.write(prov_serialization)